In [1]:
import numpy as np
from torch.nn import Sequential, Conv2d, LazyLinear, BatchNorm2d, ReLU, Flatten, Softmax, MaxPool2d
import torch

In [2]:
data = np.load('tests/00_unittest_classifier_input/train_data.npz')
X = data['X'].reshape(-1, 1, 40, 100)   #pytorch dimensions are (N, C, H, W)
y = data['y'].astype(np.int64)

In [3]:
X.shape

(1050, 1, 40, 100)

In [4]:
y.shape

(1050,)

In [5]:
y

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [6]:
h, w, c = 40, 100, 1

In [83]:
model = Sequential(
        Conv2d(in_channels=c, out_channels=32, kernel_size=3),
        BatchNorm2d(32),
        ReLU(),
        MaxPool2d(kernel_size=2),

        Conv2d(in_channels=32, out_channels=64, kernel_size=3),
        BatchNorm2d(64),
        ReLU(),
        MaxPool2d(kernel_size=2),

        Conv2d(in_channels=64, out_channels=128, kernel_size=3),
        BatchNorm2d(128),
        ReLU(),

        Flatten(),
        LazyLinear(2),
        Softmax(dim=1)
    )

C:\ProgramData\Python3_64\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [84]:
target_shape = model[:-5](torch.from_numpy(X)).shape

In [85]:
f = model(torch.from_numpy(X))#

In [86]:
f.shape

torch.Size([1050, 2])

In [87]:
cropped_model = Sequential(
        Conv2d(in_channels=c, out_channels=32, kernel_size=3),
        BatchNorm2d(32),
        ReLU(),
        MaxPool2d(kernel_size=2),

        Conv2d(in_channels=32, out_channels=64, kernel_size=3),
        BatchNorm2d(64),
        ReLU(),
        MaxPool2d(kernel_size=2),

        Conv2d(in_channels=64, out_channels=128, kernel_size=3),
        BatchNorm2d(128),
        ReLU(),
    
        Conv2d(in_channels=128, out_channels=2, kernel_size=(6, 21))    
)

In [88]:
target_shape

torch.Size([1050, 128, 6, 21])

In [89]:
# cropped_model.train()
cropped_model[-1].weight.data = model[-2].weight.data.reshape(2, target_shape[1], target_shape[2], target_shape[3]) #(2, 128, 6, 21)#[...]

In [90]:
model[-2].weight.data

tensor([[-0.0016, -0.0060, -0.0045,  ...,  0.0063, -0.0075,  0.0058],
        [-0.0036, -0.0037, -0.0043,  ...,  0.0002,  0.0053, -0.0064]])

In [91]:
cropped_model[-1].bias.data

tensor([-0.0016,  0.0002])

In [92]:
list(cropped_model[-1].parameters())[0].shape

torch.Size([2, 128, 6, 21])

In [93]:
# list(cropped_model[-1].parameters())[0] = list(model[-2].parameters())[0].reshape(2, 128, 6, 21)

In [94]:
# list(cropped_model[-1].parameters())#[1]#.shape

In [95]:
# list(model[-2].parameters())[0].reshape(2, 128, 6, 21)

In [96]:
detached

NameError: name 'detached' is not defined

In [97]:
256*6*21

32256

In [98]:
detached[0].reshape(2, 128, 6, 21)

NameError: name 'detached' is not defined

In [99]:
cropped_model[0].parameters

<bound method Module.parameters of Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))>

In [100]:
f = cropped_model(torch.from_numpy(X))

In [101]:
f.shape

torch.Size([1050, 2, 1, 1])

In [102]:
f[0]

tensor([[[-0.5517]],

        [[ 0.7637]]], grad_fn=<SelectBackward0>)

In [103]:
def get_sizes(h, w):
    src_size = np.array((h, w))
    ker_1 = np.array([3,3])
    ker_2 = np.array([6, 21])
    size_1 = src_size - ker_1 + 1
    size_2 = size_1//2 - ker_1 + 1
    size_3 = size_2//2 - ker_1 + 1
    size_4 = size_3 - ker_2 + 1
    return size_1, size_2, size_3, size_4

In [104]:
print_sizes(40, 100)

[38 98] [17 47] [ 6 21] [1 1]


In [105]:
print_sizes(220, 370)

[218 368] [107 182] [51 89] [46 69]


In [106]:
220 - 40

180

In [107]:
for i in range(0, 220):
    print(i, get_sizes(i, 370)[-1][0])

0 -9
1 -9
2 -8
3 -8
4 -8
5 -8
6 -7
7 -7
8 -7
9 -7
10 -6
11 -6
12 -6
13 -6
14 -5
15 -5
16 -5
17 -5
18 -4
19 -4
20 -4
21 -4
22 -3
23 -3
24 -3
25 -3
26 -2
27 -2
28 -2
29 -2
30 -1
31 -1
32 -1
33 -1
34 0
35 0
36 0
37 0
38 1
39 1
40 1
41 1
42 2
43 2
44 2
45 2
46 3
47 3
48 3
49 3
50 4
51 4
52 4
53 4
54 5
55 5
56 5
57 5
58 6
59 6
60 6
61 6
62 7
63 7
64 7
65 7
66 8
67 8
68 8
69 8
70 9
71 9
72 9
73 9
74 10
75 10
76 10
77 10
78 11
79 11
80 11
81 11
82 12
83 12
84 12
85 12
86 13
87 13
88 13
89 13
90 14
91 14
92 14
93 14
94 15
95 15
96 15
97 15
98 16
99 16
100 16
101 16
102 17
103 17
104 17
105 17
106 18
107 18
108 18
109 18
110 19
111 19
112 19
113 19
114 20
115 20
116 20
117 20
118 21
119 21
120 21
121 21
122 22
123 22
124 22
125 22
126 23
127 23
128 23
129 23
130 24
131 24
132 24
133 24
134 25
135 25
136 25
137 25
138 26
139 26
140 26
141 26
142 27
143 27
144 27
145 27
146 28
147 28
148 28
149 28
150 29
151 29
152 29
153 29
154 30
155 30
156 30
157 30
158 31
159 31
160 31
161 31
162 32
163 32
16

In [108]:
for j in range(0, 370):
    print(j, get_sizes(220, j)[-1][1])

0 -24
1 -24
2 -23
3 -23
4 -23
5 -23
6 -22
7 -22
8 -22
9 -22
10 -21
11 -21
12 -21
13 -21
14 -20
15 -20
16 -20
17 -20
18 -19
19 -19
20 -19
21 -19
22 -18
23 -18
24 -18
25 -18
26 -17
27 -17
28 -17
29 -17
30 -16
31 -16
32 -16
33 -16
34 -15
35 -15
36 -15
37 -15
38 -14
39 -14
40 -14
41 -14
42 -13
43 -13
44 -13
45 -13
46 -12
47 -12
48 -12
49 -12
50 -11
51 -11
52 -11
53 -11
54 -10
55 -10
56 -10
57 -10
58 -9
59 -9
60 -9
61 -9
62 -8
63 -8
64 -8
65 -8
66 -7
67 -7
68 -7
69 -7
70 -6
71 -6
72 -6
73 -6
74 -5
75 -5
76 -5
77 -5
78 -4
79 -4
80 -4
81 -4
82 -3
83 -3
84 -3
85 -3
86 -2
87 -2
88 -2
89 -2
90 -1
91 -1
92 -1
93 -1
94 0
95 0
96 0
97 0
98 1
99 1
100 1
101 1
102 2
103 2
104 2
105 2
106 3
107 3
108 3
109 3
110 4
111 4
112 4
113 4
114 5
115 5
116 5
117 5
118 6
119 6
120 6
121 6
122 7
123 7
124 7
125 7
126 8
127 8
128 8
129 8
130 9
131 9
132 9
133 9
134 10
135 10
136 10
137 10
138 11
139 11
140 11
141 11
142 12
143 12
144 12
145 12
146 13
147 13
148 13
149 13
150 14
151 14
152 14
153 14
154 15
155 15


In [109]:
for x in range(69):
    start = x*4
    end = 100 + start
    print(x, start, end)

0 0 100
1 4 104
2 8 108
3 12 112
4 16 116
5 20 120
6 24 124
7 28 128
8 32 132
9 36 136
10 40 140
11 44 144
12 48 148
13 52 152
14 56 156
15 60 160
16 64 164
17 68 168
18 72 172
19 76 176
20 80 180
21 84 184
22 88 188
23 92 192
24 96 196
25 100 200
26 104 204
27 108 208
28 112 212
29 116 216
30 120 220
31 124 224
32 128 228
33 132 232
34 136 236
35 140 240
36 144 244
37 148 248
38 152 252
39 156 256
40 160 260
41 164 264
42 168 268
43 172 272
44 176 276
45 180 280
46 184 284
47 188 288
48 192 292
49 196 296
50 200 300
51 204 304
52 208 308
53 212 312
54 216 316
55 220 320
56 224 324
57 228 328
58 232 332
59 236 336
60 240 340
61 244 344
62 248 348
63 252 352
64 256 356
65 260 360
66 264 364
67 268 368
68 272 372


In [110]:
cr_f = cropped_model(torch.from_numpy(X))

In [111]:
cr_f.shape

torch.Size([1050, 2, 1, 1])

In [112]:
del f

In [115]:
flatten = torch.flatten(cr_f, start_dim = 2, end_dim = 3)

In [117]:
flatten[:10]

tensor([[[-0.5517],
         [ 0.7637]],

        [[ 0.3442],
         [ 0.2026]],

        [[-0.1556],
         [ 0.3629]],

        [[-0.2245],
         [ 0.7665]],

        [[-0.6358],
         [ 0.8936]],

        [[-0.0892],
         [ 0.2073]],

        [[ 0.0146],
         [-0.0802]],

        [[ 0.0074],
         [-0.0374]],

        [[-0.4179],
         [ 0.0377]],

        [[-0.1566],
         [-0.1240]]], grad_fn=<SliceBackward0>)

In [148]:
mask = np.argmax(flatten.detach().numpy(), axis = 1).flatten()

In [149]:
mask[:10]

array([1, 0, 1, 1, 1, 1, 0, 0, 1, 1], dtype=int64)

In [150]:
indices = np.arange(flatten.shape[0])

In [151]:
h,w = 50, len(indices)//50

In [152]:
h,w

(50, 21)

In [153]:
indices[:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [156]:
indices[mask == 1].shape

(976,)

In [173]:
conf = flatten.detach().numpy()[mask == 1]
conf

array([[[-0.5516984 ],
        [ 0.76373994]],

       [[-0.15561241],
        [ 0.36289987]],

       [[-0.22449431],
        [ 0.7665265 ]],

       ...,

       [[-0.20409913],
        [ 1.1185802 ]],

       [[-0.49401286],
        [ 0.55669546]],

       [[-0.35936767],
        [ 0.74596524]]], dtype=float32)

In [174]:
conf[:, 1, :]

array([[ 7.63739944e-01],
       [ 3.62899870e-01],
       [ 7.66526520e-01],
       [ 8.93611372e-01],
       [ 2.07319513e-01],
       [ 3.77370901e-02],
       [-1.23993404e-01],
       [ 3.21396232e-01],
       [ 3.31634104e-01],
       [ 3.59407336e-01],
       [ 4.41279262e-01],
       [ 2.15739265e-01],
       [ 1.70399640e-02],
       [ 5.41486561e-01],
       [ 5.52991927e-01],
       [ 6.60967708e-01],
       [ 1.04295170e+00],
       [ 1.14353228e+00],
       [ 1.14899898e+00],
       [ 8.51726234e-01],
       [ 5.33390462e-01],
       [ 7.32523978e-01],
       [ 1.79386362e-01],
       [ 8.12794745e-01],
       [ 4.34202552e-01],
       [ 7.97755539e-01],
       [ 1.10022497e+00],
       [ 1.03077948e+00],
       [ 7.35082924e-01],
       [ 6.72912180e-01],
       [ 5.38375735e-01],
       [ 3.19526821e-01],
       [-4.67636138e-02],
       [ 5.98001420e-01],
       [ 4.06182185e-02],
       [ 5.04909933e-01],
       [ 6.51651800e-01],
       [ 2.35767365e-01],
       [ 5.6